In [52]:
from mlens.ensemble import SuperLearner, BlendEnsemble
import pickle
from sklearn.ensemble import VotingClassifier
import pandas as pd
import numpy as np
from scipy.optimize import minimize, fsolve
from time import time
import datetime
from xgboost import XGBRegressor

In [53]:
SEED = 21

In [54]:
train = pd.read_feather("transform/train_transform_after_preparation.feather")
pred = pd.read_feather("transform/test_transform_after_preparation.feather")

In [55]:
X_train = train.drop(['SalePrice', 'Id'], 1)
y_train = np.log1p(train['SalePrice'])

/tmp/ipykernel_4480/2050573941.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = train.drop(['SalePrice', 'Id'], 1)


In [56]:
X_pred = pred.drop('Id', 1)

/tmp/ipykernel_4480/2827053377.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_pred = pred.drop('Id', 1)


In [57]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,7,7,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,34,34,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,9,8,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,95,40,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,10,10,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


In [58]:
y_true = train['SalePrice'].values

In [59]:
y_true 

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [60]:
y_true = np.log1p(y_true)

In [61]:
y_true 

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

# Load models

In [11]:
lgb_file = "lgb/lgb_bayes_5folds_training_10folds_predict.pkl"
lasso_file = "lasso/lasso_bayes_robust_scaler.pkl"

In [12]:
with open(lasso_file, 'rb') as f:
    lasso = pickle.load(f)
    oof_lasso_load = pickle.load(f)
    sub_lasso_load = pickle.load(f)

In [13]:
with open(lgb_file, 'rb') as f:
    lgb = pickle.load(f)
    oof_lgb_load = pickle.load(f)
    sub_lgb_load = pickle.load(f)

In [14]:
lgb

LGBMRegressor(colsample_bytree=0.6375272707875022,
              learning_rate=0.04747556447678696, max_depth=26,
              min_child_weight=12.355289016132312, n_estimators=946,
              num_leaves=3, reg_lambda=2.5674370007989773)

In [15]:
oof_lgb_load

array([204660.73493536, 172631.97803027, 212034.93036911, ...,
       245423.55983916, 142918.39827003, 152169.42528783])

In [16]:
oof_lgb = np.log1p(oof_lgb_load)

In [17]:
oof_lgb

array([12.22911382, 12.05892311, 12.26451102, ..., 12.41074489,
       11.8700361 , 11.93275639])

In [18]:
sub_lgb_load

array([120515.40738419, 161359.69732025, 185182.36666844, ...,
       161033.206955  , 117733.35121203, 209463.6420758 ])

In [19]:
lasso

Lasso(alpha=0.0006109139664427438, max_iter=1028.3018415181295,
      tol=0.00038080415058498035)

In [20]:
oof_lasso_load

array([209492.92688934, 200698.04000235, 216413.34555741, ...,
       268722.44820227, 142202.00556517, 153475.77539338])

In [21]:
oof_lasso = np.log1p(oof_lasso_load)

In [22]:
oof_lasso

array([12.25245003, 12.20956175, 12.28495012, ..., 12.50143806,
       11.86501093, 11.94130453])

In [23]:
sub_lasso_load

array([118728.89320158, 154435.59318384, 184061.68757863, ...,
       167151.58039745, 118182.01099241, 229033.29091045])

In [24]:
oof = np.zeros((2, y_true.shape[0]))

In [25]:
oof[0] = oof_lgb
oof[1] = oof_lasso
oof

array([[12.22911382, 12.05892311, 12.26451102, ..., 12.41074489,
        11.8700361 , 11.93275639],
       [12.25245003, 12.20956175, 12.28495012, ..., 12.50143806,
        11.86501093, 11.94130453]])

In [26]:
def get_models():
    models = []
    models.append(lgb)
    models.append(lasso)
    return models

In [27]:
models = get_models()

# Voting Classifier

In [28]:
#clf = [rf, et, knn, svc, rg]
#eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')

# Weighted Averaging

In [29]:
from sklearn.metrics import mean_squared_error

In [30]:
def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof, axes = 1)
    #oof_blend = weights * oof
    return mean_squared_error(y_true, oof_blend)

In [31]:
"""def grad_func(weights):
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof[i], np.zeros((oof.shape[1]))
        c += weights[i] * oof[i]
        gradients[i] = 1.0 / oof.shape[0] * (a-c) * (-a)
    return gradients"""

'def grad_func(weights):\n    gradients = np.zeros(oof.shape[0])\n    for i in range(oof.shape[0]):\n        a, b, c = y_true, oof[i], np.zeros((oof.shape[1]))\n        c += weights[i] * oof[i]\n        gradients[i] = 1.0 / oof.shape[0] * (a-c) * (-a)\n    return gradients'

In [32]:
neg_mean_square_scores = {}
for i, key in enumerate(oof):
    score_oof = mean_squared_error(y_true, key)
    neg_mean_square_scores[i] = score_oof
    print(f'{i+1} CV MSE:\t', score_oof)
    print(f'{i+1} CV RMSE:\t', np.sqrt(score_oof))

1 CV MSE:	 0.013117135458378159
1 CV RMSE:	 0.11453006355703362
2 CV MSE:	 0.012508204111339691
2 CV RMSE:	 0.11184008275810463


In [33]:
init_guess = [1 / oof.shape[0]] * oof.shape[0]
init_guess

[0.5, 0.5]

In [34]:
bnds = [(0, 1) for _ in range(oof.shape[0])]
bnds

[(0, 1), (0, 1)]

In [35]:
oof

array([[12.22911382, 12.05892311, 12.26451102, ..., 12.41074489,
        11.8700361 , 11.93275639],
       [12.25245003, 12.20956175, 12.28495012, ..., 12.50143806,
        11.86501093, 11.94130453]])

In [36]:
oof_blend2 = np.tensordot(init_guess, oof, axes = 1)
oof_blend2

array([12.24078193, 12.13424243, 12.27473057, ..., 12.45609147,
       11.86752352, 11.93703046])

In [40]:
tol = 1e-8

In [41]:
# tol = 1e-10
init_guess = [1 / oof.shape[0]] * oof.shape[0]
bnds = [(0, 1) for _ in range(oof.shape[0])]
'''cons = {'type': 'eq', 
        'fun': lambda x: np.sum(x) - 1, 
        'jac': lambda x: [1] * len(x)}
'''
print('Inital Blend OOF:', func_numpy_metric(init_guess))
start_time = time()
res_scipy = minimize(fun = func_numpy_metric, 
                     x0 = init_guess, 
                     method = 'nelder-mead', 
                     #jac = grad_func, # grad_func 
                     bounds = bnds, 
                     #constraints = cons, 
                     #tol = tol,
                     options={'xatol': tol, 'disp': True})
print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] Optimised Blend OOF:', res_scipy.fun)
print('RMSE Optimised Blend OOF:', np.sqrt(res_scipy.fun))
print('Optimised Weights:', res_scipy.x)

Inital Blend OOF: 0.011942046954487799
Optimization terminated successfully.
         Current function value: 0.011915
         Iterations: 61
         Function evaluations: 122
[00:00] Optimised Blend OOF: 0.011915244158625311
RMSE Optimised Blend OOF: 0.10915697027045644
Optimised Weights: [0.41253294 0.58743138]


In [42]:
print('Check the sum of all weights:', np.sum(res_scipy.x))
if np.sum(res_scipy.x) - 1 <= tol:
    print('Great! The sum of all weights equals to 1!')
else:
    print('Manual adjustion is needed to modify the weights.')

Check the sum of all weights: 0.9999643277266199
Great! The sum of all weights equals to 1!


# Stacking

In [62]:
from sklearn.linear_model import LinearRegression
from mlens.metrics import make_scorer
from sklearn.metrics import mean_squared_error

In [63]:
def scoring(y_true, y_pred):
    return -mean_squared_error(y_true, y_pred)

In [64]:
stacking = SuperLearner(scorer=scoring, folds=10, shuffle=True, sample_size=len(X_train), random_state=SEED, verbose=1)

In [65]:
stacking.add(models)

SuperLearner(array_check=None, backend=None, folds=10,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=10, raise_on_ex...c3d64c0>)],
   n_jobs=-1, name='group-0', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f42ac3d64c0>, shuffle=True,
       verbose=1)

In [66]:
stacking.add_meta(XGBRegressor())

SuperLearner(array_check=None, backend=None, folds=10,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=10, raise_on_ex...c3d64c0>)],
   n_jobs=-1, name='group-1', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f42ac3d64c0>, shuffle=True,
       verbose=1)

In [67]:
X_train.to_numpy()

array([[6.000e+01, 6.500e+01, 8.450e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+01, 8.000e+01, 9.600e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [6.000e+01, 6.800e+01, 1.125e+04, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [7.000e+01, 6.600e+01, 9.042e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+01, 6.800e+01, 9.717e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+01, 7.500e+01, 9.937e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00]])

In [68]:
y_train.to_numpy()

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

In [69]:
stacking.fit(X_train.to_numpy(), y_train.to_numpy())


Fitting 2 layers


/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.702e+00, tolerance: 7.963e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.437e+00, tolerance: 7.812e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.496e+00,

Fit complete                        | 00:00:05


SuperLearner(array_check=None, backend=None, folds=10,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=10, raise_on_ex...c3d64c0>)],
   n_jobs=-1, name='group-1', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f42ac3d64c0>, shuffle=True,
       verbose=1)

In [70]:
mse = mean_squared_error(y_train.to_numpy(), stacking.predict(X_train.to_numpy()))


Predicting 2 layers
Predict complete                    | 00:00:00


In [71]:
mse

0.009828809020354616

In [72]:
rmse = np.sqrt(mse)
rmse

0.09914035011212446

In [73]:
stacking_pred = np.expm1(stacking.predict(X_pred.to_numpy()))


Predicting 2 layers
Predict complete                    | 00:00:00


In [74]:
stacking_pred

array([119453.75, 151543.36, 177670.95, ..., 160467.23, 121867.65,
       254489.94], dtype=float32)

# Blending

In [76]:
blending = BlendEnsemble(test_size=0.5, shuffle=True, scorer=scoring, sample_size=len(X_train), random_state=SEED, verbose=1)

In [77]:
blending.add(models)

BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=T...c3d64c0>)],
   n_jobs=-1, name='group-2', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f42ac3d64c0>, shuffle=True,
       test_size=0.5, verbose=1)

In [78]:
blending.add_meta(XGBRegressor())

BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=T...c3d64c0>)],
   n_jobs=-1, name='group-3', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f42ac3d64c0>, shuffle=True,
       test_size=0.5, verbose=1)

In [79]:
blending.fit(X_train.to_numpy(), y_train.to_numpy())


Fitting 2 layers


/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.454e+00, tolerance: 4.611e-02
  model = cd_fast.enet_coordinate_descent(
/home/roma/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.351e+00, tolerance: 8.692e-02
  model = cd_fast.enet_coordinate_descent(


Fit complete                        | 00:00:01


BlendEnsemble(array_check=None, backend=None,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=5327, shuffle=True,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=BlendIndex(X=None, raise_on_exception=T...c3d64c0>)],
   n_jobs=-1, name='group-3', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=21, sample_size=1456,
       scorer=<function scoring at 0x7f42ac3d64c0>, shuffle=True,
       test_size=0.5, verbose=1)

In [80]:
mse = mean_squared_error(y_train.to_numpy(), blending.predict(X_train.to_numpy()))
mse


Predicting 2 layers
Predict complete                    | 00:00:00


0.014238851757953995

In [81]:
rmse = np.sqrt(mse)
rmse

0.11932665987931614

In [82]:
blending_pred = np.expm1(stacking.predict(X_pred.to_numpy()))


Predicting 2 layers
Predict complete                    | 00:00:00


In [83]:
blending_pred

array([119453.75, 151543.36, 177670.95, ..., 160467.23, 121867.65,
       254489.94], dtype=float32)

# Final Submission

In [46]:
0.41253294 + 0.58743138

0.9999643199999999

In [85]:
# LB 0.14157
ens = stacking_pred
submission = pd.DataFrame()
submission['Id'] = pred['Id']
submission["SalePrice"] = ens
submission.to_csv("submit_ensemble.csv", index=False)